Evaluar $$\int_{1}^2 \frac{1}{x^2}dx$$usando *cuadratura de Gauss-Legendre* con 2 y 3 puntos.

Para un número $n$ de puntos se tiene que 
$$
\int_a^bf(x)dx = \frac{1}{2}(b-a)\sum_1^n w_kf(\frac{1}{2}[t_k(b-a)+a+b]), \quad k=1, \ldots, n.
$$
En este caso, $a=1, b=2$, por lo que 
$$
\int_1^2f(x)dx = \frac{1}{2}\sum_1^n w_kf(\frac{1}{2}[t_k+3]), \quad k=1, \ldots, n.
$$
Cada peso $w_k$ y cada $t_k$ están dados por 
$$
w_k=\frac{2}{(1-x_k^2)[p'_n(x_k)]^2}, \quad t_k = \frac{2x_k-a-b}{b-a}=2x_k-3
$$
En donde cada $x_k$ hace referencia a cada una de las $n$ raíces del $n$-ésimo polinomio de Legendre.
Se implementa a continuación el código para 2 y 3 puntos.

In [11]:
import numpy as np
import sympy as sym
from tqdm import tqdm

In [12]:
def GetLegendre(n):
    '''
    Calcula el n-ésimo polinomio de Legendre haciendo uso de la
    fórmula de rodrigues.
    Parámetros:
        n: grado del polinomio de Legendre que se quiere estimar.
    Retornos:
        plambda: polinomio de Legendre estimado convertido en función mediante lambdify.
        p = polinomio simbolizado de Legendre.
    '''
    x = sym.Symbol('x', Real=True)
    y = sym.Symbol('y', Real=True)

    y = (x**2-1)**n
    p = sym.diff(y, x, n)/(2**n*np.math.factorial(n)) #derivada n-esima de y con respecto a x

    plambda = sym.lambdify([x], p, 'numpy')
    return plambda, p


In [13]:
def GetRootsLegendre(f, df, xn, itmax=1000, precision = 1e-5):
    '''
    Calcula las raíces de un polinomio de Legendre con el método de Newton-Raphson
    Parámetros:
        f: polinomio de Legendre.
        df: derivada primera del polinomio f.
        xn: punto de partida para el método Newton-Raphson.
    Retornos:
        xn: raíz encontrada
    '''
    error = 1
    it = 0
    while error>precision and it<itmax:
        try:
            xn1 = xn - f(xn)/df(xn)
            error = np.abs(f(xn)/df(xn))
        except ZeroDivisionError:
            print('División por cero')
        xn = xn1
        it += 1
    if it == itmax:
        return False
    else:
        return xn

In [14]:
def GetAllRoots(n, tol=6):
    '''
    Retorna las n raíces de un polinomio de grado n de Legendre.
    Parámetros:
        n: grado del polinomio.
    Retornos:
        roots: lista con las raíces encontradas, ordenadas de menor a mayor.
    '''
    x_trial = np.linspace(-1, 1, 200)
    roots = np.array([])
    poly,p = GetLegendre(n)
    x = sym.Symbol('x', Real=True)
    df = sym.diff(p, x, 1)
    df = sym.lambdify([x], df, 'numpy')
    for i in tqdm(x_trial):
        root = GetRootsLegendre(poly, df, i)
        croot = np.round(root, tol)
        if croot not in roots:
            roots = np.append(roots, croot)
    roots.sort()
    return roots

In [15]:
def GetWeights(Roots, dpoly):
    '''
    Calcula los pesos asociados a raíces de Legendre para la Cuadratura de Gauss-Legendre.
    Parámetros:
        Roots: lista con las raíces del polinomio de Legendre.
    Retornos:
        Weights: Lista de pesos asociados a cada raíz en Roots, a la i-esima entrada de Roots
        le corresponde el peso i-esimo en Weights.
    '''
    x = sym.Symbol('x', Real=True)
    Weights = []
    dpn = sym.lambdify([x], dpoly,'numpy')
    for r in Roots:
        Weights.append(2/((1-r**2)*dpn(r)**2))

    return Weights

In [16]:
a=1
b=2
x = sym.Symbol('x', Real=True)
f = lambda x: 1/x**2

In [17]:
for i in [2, 3]:
    Legendre_poly = GetLegendre(i)[1]
    Der_Legendre = sym.diff(Legendre_poly, x, 1)
    Roots = GetAllRoots(i)
    Weights = GetWeights(Roots, Der_Legendre)
    t = 0.5*((b-a)*Roots + a + b)
    Integral = 0.5*(b-a)*np.sum(Weights*f(t))
    print('Para ' + str(i) + ' puntos la integral estimada es '+ str(Integral))

100%|██████████| 200/200 [00:00<00:00, 11807.79it/s]

Para 2 puntos la integral estimada es 0.4970415996577904
Para 3 puntos la integral estimada es 0.4998736894004501
